In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from torchtext import data
from torchtext import datasets

import numpy as np
from sklearn.model_selection import KFold 

import time
import random

In [ ]:
SEED = 1234

# We have looked at https://github.com/bentrevett/pytorch-sentiment-analysis for setting the various values of TEXT and LABEL as we have not worked with this before till now.

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)                                   #split the data into training and test data
train_data, val_data= train_data.split(split_ratio=0.8, random_state=random.seed(SEED))     #split training data into training and validation data

print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(val_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 20000
Number of validation examples: 5000
Number of testing examples: 25000


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(vars(train_data.examples[0]))

{'text': ['...', 'but', 'a', 'lousy', 'film', '.', 'As', 'Maltin', 'says', 'this', 'was', 'Christopher', 'Lee', "'s", 'attempt', 'to', 'make', 'a', 'serious', 'horror', 'film', '.', 'Well', ',', 'it', 'is', 'serious', '...', 'TOO', 'serious', '!', 'The', 'plot', 'is', 'silly', 'and', 'slow', '(', 'something', 'about', 'old', 'people', 'inhabiting', 'the', 'bodies', 'of', 'young', 'children', 'to', 'achieve', 'immortality)', '...', 'the', 'film', 'is', 'all', 'talk', 'talk', 'talk', 'talk', 'talk', 'talk', 'talk', 'about', 'the', 'same', 'things', 'over', 'and', 'over', 'again', '.', 'I', 'actually', 'dozed', 'off', 'a', 'few', 'times', '!', 'The', 'film', 'is', 'sooooo', 'dull', '!', 'The', 'cast', 'sleepwalks', 'through', 'this', 'with', 'the', 'sole', 'exceptions', 'of', 'Peter', 'Cushing', 'and', 'Lee', '...', 'but', 'this', 'was', 'probably', 'a', 'labor', 'of', 'love', 'for', 'both', '(', 'they', 'often', 'complained', 'about', 'horror', 'movies', 'being', 'too', 'violent', '...',

In [ ]:
MAX_VOCAB_SIZE = 50000 #consider only the top 50000 used words to build vocabulary/embeddings

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 50002
Unique tokens in LABEL vocabulary: 2


In [ ]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 231843), (',', 219895), ('.', 189748), ('and', 125166), ('a', 124925), ('of', 115411), ('to', 107330), ('is', 87251), ('in', 70025), ('I', 62108), ('it', 60994), ('that', 56178), ('"', 50106), ("'s", 49515), ('this', 48357), ('-', 42046), ('/><br', 40789), ('was', 39878), ('as', 34817), ('with', 34347)]


In [ ]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [ ]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f4237351510>, {'neg': 0, 'pos': 1})


In [ ]:
BATCH_SIZE = 64

train_iterator, val_iterator, test_iterator = data.BucketIterator.splits(         #make the iterators for train,test,val
    (train_data, val_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [ ]:
#This model has been built from scratch without using the links provided in the doc. The basic framework is taken from https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

class BiLSTM(nn.Module):
    def __init__(self,input_dim,embedding_dim,hidden_dim,output_dim,n_layers,bidirectional,dropout,pad_idx):
        super().__init__()

        self.embedding=nn.Embedding(input_dim,embedding_dim,padding_idx=pad_idx)

        self.lstm=nn.LSTM(embedding_dim, hidden_dim,num_layers=n_layers,bidirectional=bidirectional,dropout=dropout if n_layers>1 else 0)

        #self.gru=nn.GRU(embedding_dim, hidden_dim,num_layers=n_layers,bidirectional=bidirectional,dropout=dropout if n_layers>1 else 0)

        self.fc=nn.Linear(hidden_dim, output_dim)

        self.dropout=nn.Dropout(dropout)

    def forward(self,text,text_lengths):
        embed=self.embedding(text)
        #text_lengths = torch.as_tensor(text_lengths, dtype=torch.int64, device='cpu')
        #packed_embedded = nn.utils.rnn.pack_padded_sequence(embed, text_lengths) uncomment to introduce padding of sequence
        #packed_output, (hidden, cell) = self.lstm(packed_embedded)               uncomment to train lstm with padding
        #packed_output, (hidden, cell) = self.gru(packed_embedded)                uncomment to train gru with padding
        output, (hidden,cell) = self.lstm(embed)  
        #output, hidden = self.gru(embed)                                         uncomment for normal gru
        #output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output) uncomment if using padding
        #print(hidden.shape)
        hidden=self.dropout(hidden)
        #hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)) uncomment if using bidirectional lstm only (not gru)
        return self.fc(hidden)

In [ ]:
#pretrained_embeddings = TEXT.vocab.vectors
#print(pretrained_embeddings.shape)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM=1
#OUTPUT_DIM = len(LABEL.vocab)
N_LAYERS = 1
BIDIRECTIONAL = False #keep true if using bidirectional GRU/LSTM
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
print(INPUT_DIM)
print(OUTPUT_DIM)

1
50002
1


In [ ]:
model=BiLSTM(INPUT_DIM,EMBEDDING_DIM,HIDDEN_DIM,OUTPUT_DIM,N_LAYERS,BIDIRECTIONAL,DROPOUT,PAD_IDX)  #instantiate the model
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)                                                #used Adam optimizer
#loss_function = nn.NLLLoss()
loss_function=nn.BCEWithLogitsLoss()                                                                #used a variation of binary cross entropy loss
loss_function.to(device)

BCEWithLogitsLoss()

In [ ]:
#model.embedding.weight.data.copy_(pretrained_embeddings)

In [ ]:
# We have used this function directly from https://github.com/bentrevett/pytorch-sentiment-analysis but there is nothing new about this as this is a standard implementation
def binary_accuracy(preds, y):      #returns the probability of correctly classified data
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
!nvidia-smi

Thu Nov 12 10:12:03 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    32W / 250W |    883MiB / 16280MiB |      6%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
num_epochs=20

trainloss=[]
trainacc=[]
valacc=[]
valloss=[]

#We have used the training and testing loops based on the pytorch link mentioned before. Few changes are there for dimension matching etc.

for epoch in range(num_epochs):
  start_time=time.time()
  model.train()
################################################ TRAINING ################################################
  train_acc=0
  train_loss=0
  for batch in train_iterator:
    model.zero_grad()
    #print(batch.text.shape)
    text, text_lengths= batch.text
    #print(sentiment.shape)
    sentiment=model(text, text_lengths).squeeze(1)
    sentiment=torch.reshape(sentiment,(len(batch.label),))
    #print(sentiment.shape)
    #print(batch.label.shape)
    loss=loss_function(sentiment,batch.label)
    loss.backward()
    optimizer.step()
    train_loss=train_loss+loss.item()
    #_,indices=torch.max(sentiment,1)
    #train_acc=train_acc+torch.mean(torch.tensor(batch.label == indices, dtype=torch.float))
    train_acc=train_acc+binary_accuracy(sentiment, batch.label)

  train_loss=train_loss/len(train_iterator)
  train_acc=train_acc/len(train_iterator)
  trainloss.append(float(train_loss))
  trainacc.append(float(train_acc))

  val_acc=0
  val_loss=0
  model.eval()

############################################################# VALIDATION/TEST ###############################################

  with torch.no_grad():
    for batch in val_iterator:
      text, text_lengths= batch.text
      text_lengths = torch.as_tensor(text_lengths, dtype=torch.int64, device='cpu')
      prediction=model(text, text_lengths).squeeze(1)
      prediction=torch.reshape(prediction,(len(batch.label),))
      val_loss=val_loss+loss_function(prediction,batch.label).item()
      #_,indices=torch.max(prediction,1)
      #val_acc=val_acc+torch.mean(torch.tensor(batch.label == indices, dtype=torch.float))
      val_acc=val_acc+binary_accuracy(prediction, batch.label)

  val_loss=val_loss/len(val_iterator)
  val_acc=val_acc/len(val_iterator)
  valloss.append(float(val_loss))
  valacc.append(float(val_acc))

  end_time=time.time()

  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
  print(f'\t Val. Loss: {val_loss:.3f} |  Val. Acc: {val_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 10s
	Train Loss: 0.676 | Train Acc: 57.55%
	 Val. Loss: 0.745 |  Val. Acc: 57.95%
Epoch: 02 | Epoch Time: 0m 10s
	Train Loss: 0.652 | Train Acc: 61.26%
	 Val. Loss: 0.687 |  Val. Acc: 55.24%
Epoch: 03 | Epoch Time: 0m 10s
	Train Loss: 0.606 | Train Acc: 67.65%
	 Val. Loss: 0.648 |  Val. Acc: 61.17%
Epoch: 04 | Epoch Time: 0m 10s
	Train Loss: 0.536 | Train Acc: 73.83%
	 Val. Loss: 0.674 |  Val. Acc: 59.63%
Epoch: 05 | Epoch Time: 0m 10s
	Train Loss: 0.640 | Train Acc: 62.84%
	 Val. Loss: 0.514 |  Val. Acc: 77.10%
Epoch: 06 | Epoch Time: 0m 10s
	Train Loss: 0.510 | Train Acc: 76.58%
	 Val. Loss: 0.402 |  Val. Acc: 83.58%
Epoch: 07 | Epoch Time: 0m 10s
	Train Loss: 0.307 | Train Acc: 88.21%
	 Val. Loss: 0.335 |  Val. Acc: 86.06%
Epoch: 08 | Epoch Time: 0m 10s
	Train Loss: 0.202 | Train Acc: 92.77%
	 Val. Loss: 0.441 |  Val. Acc: 80.91%
Epoch: 09 | Epoch Time: 0m 10s
	Train Loss: 0.153 | Train Acc: 94.99%
	 Val. Loss: 0.363 |  Val. Acc: 86.23%
Epoch: 10 | Epoch T

In [ ]:
import csv

rows=zip(trainloss,trainacc,valloss,valacc)
with open("lstm.csv", "w") as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)

In [ ]:
# References
# https://github.com/bentrevett/pytorch-sentiment-analysis
# https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html